In [1]:
import os
import matplotlib.pyplot as plt
import scipy.misc
import sklearn.feature_extraction
import numpy as np
import scipy.ndimage
from skimage import measure, io
from skimage import transform
import train_yuv 
import skimage
import json
import datetime
import pickle

save_results = r'C:\Users\buggyr\Mosaic_Experiments\models'
load_training = r'C:\Users\buggyr\Mosaic_Experiments\data\external\Training Images 3'


Using TensorFlow backend.


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Activation, UpSampling2D,Reshape
from keras.optimizers import Adadelta, Nadam, RMSprop
from keras.models import load_model

In [3]:
#Define Network 
f1 = 64
f2 = 32
f3 = 32
ff = 3
k1 = (9,9)
k2 = (1,1)
k3 = (5,5)

model = Sequential()

model.add(Conv2D(f1,k1, padding = 'same', input_shape=(None, None ,4), activation = 'relu'))

model.add(UpSampling2D(size=(2,2)))

model.add(Conv2D(f2,k2, padding = 'same', activation='relu'))

model.add(Conv2D(f3,k3, padding = 'same',activation = 'relu'))

model.add(Conv2D(ff,k3, padding = 'same',activation = 'tanh'))

model.add(Conv2D(ff,1, padding = 'same', kernel_initializer=train_yuv.yuv_init, 
                 input_shape = (None, None, 3), trainable = False))

modsum = model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, None, None, 64)    20800     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 32)    2080      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 32)    25632     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, None, None, 3)     2403      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, None, None, 3)     12        
Total params: 50,927
Trainable params: 50,915
Non-trainable params: 12
_________________________________________________________________


In [ ]:
#Initialise Training 
now=datetime.datetime.now()
save_file=os.path.join(save_results,now.strftime("%Y-%m-%d %H-%M"))
os.mkdir(save_file)
save_pred = os.path.join(save_file,'Epoch_Predictions')
os.mkdir(save_pred)
save_test = os.path.join(save_file,'Test_Results')
os.mkdir(save_test)


with open(os.path.join(save_file,'Model_Summary.txt'),'w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))
    
#rmsprop = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)
optimizer_func = Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
loss_func='mse'

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
epoch_predict = train_yuv.Save_predictions(save_pred)


model.compile(optimizer=optimizer_func,loss=loss_func)

In [ ]:
#Train
ls = len(os.listdir(load_training))

                            #train_dir, patch_size, batch_size
train_generator = train_yuv.train_generator_yuv(load_training,32,32)
history = model.fit_generator(generator = train_generator,steps_per_epoch=3*fls,
                             epochs = 200,callbacks = [tbCallBack,csv_logger,epoch_predict])
print(history.history)

model.save(os.path.join(save_file,'DeMos_mod.h5'))

In [ ]:
#Test Kodak
import cv2
kodak_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak'
ls = len(os.listdir(kodak_dir))

kodak_generator = train_yuv.predict_generator_yuv(kodak_dir)

# k_pred = model.predict_generator(kodak_generator, steps = ls)

def pred_generator(k_gen,steps):
    
    psnr = 0
    ssim = 0
    
    for i in range(steps):
        imgs_pred = next(k_gen)

        pred_img = (model.predict(imgs_pred[0],batch_size=1))[0]
        pred_img = ((pred_img+1)/127.5)
        orig_imgs = imgs_pred[1]
        
        cv2.imshow('image',pred_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        psnr = psnr + measure.compare_psnr(pred_img,orig_imgs)
        ssim= ssim + measure.compare_ssim(pred_img,orig_imgs,multichannel=True)
    psnr_K_avg = psnr/steps
    ssim_K_avg = ssim/steps
    print('Kodak PSNR Average: '+str(psnr_K_avg))
    print('Kodak SSIM Average: '+str(ssim_K_avg))
    
    return
    
pred_generator(kodak_generator,ls)

In [57]:
import importlib
importlib.reload(train_yuv)

<module 'train_yuv' from 'C:\\Users\\buggyr\\Mosaic_Experiments\\src\\train_yuv.py'>

In [ ]:
ptch_sizes = [32, 64, 128]
for ptch_size in ptch_sizes:
    print(str(ptch_size))
    
    now=datetime.datetime.now()
    save_file=os.path.join(save_results,now.strftime("%Y-%m-%d %H-%M"))
    os.mkdir(save_file)
    save_pred = os.path.join(save_file,'Epoch_Predictions')
    os.mkdir(save_pred)

    with open(os.path.join(save_file,'Model_Summary.txt'),'w') as fh:
        # Pass the file handle in as a lambda function to make it callable
        model.summary(print_fn=lambda x: fh.write(x + '\n'))

    #rmsprop = RMSprop(lr=0.0005, rho=0.9, epsilon=1e-08, decay=0.0)
    optimizer_func = Nadam(lr=0.00002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    loss_func='mse'

    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    tbCallBack = keras.callbacks.TensorBoard(log_dir=os.path.join(save_file,'TNSR_BRD'), histogram_freq=0, write_graph=True, write_images=True)
    csv_logger = keras.callbacks.CSVLogger(os.path.join(save_file,'training.log'), separator=',', append=False)
    epoch_predict = train_yuv.Save_predictions(save_pred)


    model.compile(optimizer=optimizer_func,loss=loss_func)

    #train_r.train_seq(load_training,32,40)
    fls = len(os.listdir(load_training))

                               #train_dir, patch_size, batch_size
    train_generator = train_yuv.train_generator_yuv(load_training,ptch_size,32)
    history = model.fit_generator(generator = train_generator,steps_per_epoch=3*fls,
                                 epochs = 100,callbacks = [tbCallBack,csv_logger,epoch_predict])

    model.save(os.path.join(save_file,'DeMos_mod.h5'))

In [23]:
model = keras.models.load_model(r'C:\Users\buggyr\Mosaic_Experiments\models\2017-11-18 13-03 YUV_ptch128\DeMos_mod.h5')

ValueError: Unknown initializer: yuv_init

In [4]:
model.load_weights(r'C:\Users\buggyr\Mosaic_Experiments\models\2017-11-18 13-03 YUV_ptch128\DeMos_mod.h5')

In [6]:
#Test Kodak
data ={}

kodak_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\Kodak'
ls = len(os.listdir(kodak_dir))

kodak_generator = train_yuv.predict_generator_yuv(kodak_dir)

# k_pred = model.predict_generator(kodak_generator, steps = ls)

res_Kodak = train_yuv.predict_generator(model,kodak_generator,ls,data,5,save_test)

data['Kodak_IMGS_PSNR'] = res_Kodak[0]
data['Kodak_IMGS_SSIM'] = res_Kodak[1]
data['Kodak_AVG_PSNR']  = res_Kodak[2]
data['Kodak_AVG_SSIM']  = res_Kodak[3]

NameError: name 'save_test' is not defined

In [5]:
#Test McManus
McM_dir = r'C:\Users\buggyr\Mosaic_Experiments\data\interim\McM'
ls = len(os.listdir(McM_dir))

McM_generator = train_yuv.predict_generator_yuv(McM_dir)

# k_pred = model.predict_generator(kodak_generator, steps = ls)

res_McM = train_yuv.predict_generator(model,McM_generator,ls,data,5,save_test)

data['McM_IMGS_PSNR'] = res_McM[0]
data['McM_IMGS_SSIM'] = res_McM[1]
data['McM_AVG_PSNR']  = res_McM[2]
data['McM_AVG_SSIM']  = res_McM[3]

NameError: name 'data' is not defined

In [60]:
#Write Results
data['Parameters'] = {
    'Loss Function': loss_func,
    'Optimizer':str(type(optimizer_func))
}
data['Training Set'] = {
    'Training Path': load_training,
}

with open(os.path.join(save_file,'results.txt'), 'w') as outfile:  
    json.dump(data, outfile,indent=4)

In [37]:
save_file = r'C:\Users\buggyr\Mosaic_Experiments\models\2017-11-18 13-03 YUV_ptch128'
save_test = os.path.join(save_file,'Test_Results')
os.mkdir(save_test)